## XBRL US API - Python example  
This notebook contains example Python code to use the XBRL US API (https://xbrl.us/home/use/xbrl-api/)    
  
**Made by:** [Ties de Kok](https://www.tiesdekok.com)

## Imports

In [1]:
import os, re, sys, json
import requests

In [2]:
import pandas as pd
import numpy as np

## Api documentation

https://xbrlus.github.io/xbrl-api/#/Facts/getFactDetails

## Generate an access token
The access token will expire after 60 minutes.

**Normally you would use the following code with your own credentials:**    
You can obtain credentials here: https://xbrl.us/home/use/xbrl-api-community/#provisioning

```
endpoint = 'https://api.xbrl.us'
endpoint_auth = endpoint + '/oauth2/token'

with open('login_cred.json', 'r') as f:
    login_cred = json.loads(f.read())
    
body_auth = {'grant_type' : 'password', 
            'client_id' : login_cred['client_id'], 
            'client_secret' : login_cred['client_secret'], 
            'username' : login_cred['username'] ,
            'password' : login_cred['password'],
            'platform' : 'pc'}
res = requests.post(endpoint_auth, data=body_auth)
auth_json = res.json()
access_token = auth_json['access_token']
```

### However, for this demo we will use temporary credentials:

In [3]:
access_token = requests.get('https://tdekok-xbrlapi.builtwithdark.com/gettoken').text.replace('"', "")

## Make a query

### Define the fields you would like returned

In [4]:
fields = ['entity.cik',
         'entity.name.sort(ASC)',
         'dts.id',
         'fact.id',
         'report.filing-date',
         'period.fiscal-year',
         'report.type',
         'concept.local-name',
         'dimensions.count',
         'dimension.local-name.sort(ASC)',
         'member.local-name',
         'fact.value',
         'unit',
         'fact.ultimus-index',
         'dimension.namespace',
         'member.namespace',
         'fact.offset(0)'
         ]

### Define the XBRL elements (tags) you'd like 

In [5]:
XBRL_Elements = ['IntangibleAssetsNetExcludingGoodwill',
                 'FiniteLivedIntangibleAssetsNet',
                 'IndefiniteLivedIntangibleAssetsExcludingGoodwill'
                ]

### Define the companies you'd like

In [6]:
companies_cik = ['0000789019', ## Microsoft (MSFT)
                 '0001018724', ## Amazon (AMZN)
                 '0001652044', ## Alphabet (GOOG)
                ]

### Define the years you'd like

In [7]:
years = ['2019']

### Automatic: assemble API parameters

In [8]:
params = {'concept.local-name': ','.join(XBRL_Elements),
         'fact.has-dimensions': 'TRUE',
         'report.type': '10-K,10-K/A',
         'period.fiscal-period': 'Y',
         'period.fiscal-year': ','.join(years),
         'unit': 'USD',
         'fields': ','.join(fields),
         'entity.cik': ','.join(companies_cik)
         }

In [9]:
search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'

In [10]:
res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})

In [11]:
res_json = res.json()

In [13]:
res_df = pd.DataFrame(res_json['data'])

In [14]:
res_df

,entity.cik,entity.name,dts.id,fact.id,report.filing-date,period.fiscal-year,report.type,concept.local-name,dimensions.count,dimension.local-name,member.local-name,fact.value,unit,fact.ultimus-index,dimension.namespace,member.namespace
0,0001652044,Alphabet Inc.,362758,221624796,2020-02-04,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,TradenamesAndOtherMember,353000000,USD,2,http://fasb.org/us-gaap/2019-01-31,http://www.google.com/20191231
1,0001652044,Alphabet Inc.,362758,221624695,2020-02-04,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,CustomerRelationshipsMember,224000000,USD,2,http://fasb.org/us-gaap/2019-01-31,http://fasb.org/us-gaap/2019-01-31
2,0001652044,Alphabet Inc.,362758,221625088,2020-02-04,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,PatentsAndDevelopedTechnologyMember,1402000000,USD,2,http://fasb.org/us-gaap/2019-01-31,http://www.google.com/20191231
3,0001018724,"AMAZON.COM, INC.",362382,221546174,2020-01-31,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,TechnologyBasedIntangibleAssetsMember,534000000,USD,1,http://fasb.org/us-gaap/2019-01-31,http://fasb.org/us-gaap/2019-01-31
4,0001018724,"AMAZON.COM, INC.",362382,221545329,2020-01-31,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,MarketingRelatedIntangibleAssetsMember,1963000000,USD,1,http://fasb.org/us-gaap/2019-01-31,http://fasb.org/us-gaap/2019-01-31
5,0001018724,"AMAZON.COM, INC.",362382,221545455,2020-01-31,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,CustomerRelatedIntangibleAssetsMember,152000000,USD,1,http://fasb.org/us-gaap/2019-01-31,http://fasb.org/us-gaap/2019-01-31
6,0001018724,"AMAZON.COM, INC.",362382,221546415,2020-01-31,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,ContractBasedIntangibleAssetsMember,1400000000,USD,1,http://fasb.org/us-gaap/2019-01-31,http://fasb.org/us-gaap/2019-01-31
7,0000789019,MICROSOFT CORPORATION,334516,209185196,2019-08-01,2019,10-K,FiniteLivedIntangibleAssetsNet,1,FiniteLivedIntangibleAssetsByMajorClassAxis,CustomerRelationshipsMember,2924000000,USD,4,http://fasb.org/us-gaap/2018-01-31,http://fasb.org/us-gaap/2018-01-31
8,0000789019,MICROSOFT CORPORATION,334516,209184179,2019-08-01,2019,10-K,FiniteLivedIntangibleAssetsNet,1,FiniteLivedIntangibleAssetsByMajorClassAxis,TechnologyBasedIntangibleAssetsMember,1920000000,USD,4,http://fasb.org/us-gaap/2018-01-31,http://fasb.org/us-gaap/2018-01-31
9,0000789019,MICROSOFT CORPORATION,334516,209184767,2019-08-01,2019,10-K,FiniteLivedIntangibleAssetsNet,1,FiniteLivedIntangibleAssetsByMajorClassAxis,MarketingRelatedIntangibleAssetsMember,2838000000,USD,4,http://fasb.org/us-gaap/2018-01-31,http://fasb.org/us-gaap/2018-01-31
